In [ ]:
from snowflake.snowpark import Session
import snowflake.snowpark.types as types

In [ ]:
session_params = {
    "account": "<account>",
    "user": "<username>",
    "password": "<password>",
    "role": "<role-name>",
    "warehouse": "<warehouse-name>",
    "database": "<database-name>",
    "schema": "<schema-name>"
}
session = Session.builder.configs(session_params).create()

In [ ]:
session.sql("""
	CREATE WAREHOUSE IF NOT EXISTS
	COMPUTE_WH WITH WAREHOUSE_SIZE='X-SMALL'
""").collect()
session.sql("""
	CREATE DATABASE IF NOT EXISTS TESTING_DB
""").collect()
session.sql("""
	CREATE SCHEMA IF NOT EXISTS TESTING_DB.MY_TEST_SCHEMA
""").collect()
session.sql("""
	CREATE STAGE IF NOT EXISTS
	TESTING_DB.MY_TEST_SCHEMA.MY_FIRST_STAGE
""").collect()

In [ ]:
session.use_database("TESTING_DB")
session.use_schema("MY_TEST_SCHEMA")

In [ ]:
info = session.sql("""
	select current_warehouse(),
	current_database(),
	current_schema(),
	current_user(),
	current_role()
""").collect()

In [ ]:
session.file.put('<dir>/file1.csv', 'MY_FIRST_STAGE')
session.file.put('<dir>/file2.txt', 'MY_FIRST_STAGE')
session.file.put('<dir>/file3.parquet', 'MY_FIRST_STAGE')

In [ ]:
file1_schema = types.StructType(
    [
        types.StructField("ID", types.IntegerType()),
        types.StructField("EXAMPLE_COL_1", types.IntegerType()),
        types.StructField("EXAMPLE_COL_2", types.StringType()),
        types.StructField("EXAMPLE_COL_3", types.StringType()),
        types.StructField("EXAMPLE_COL_4", types.IntegerType()),
        types.StructField("EXAMPLE_COL_5", types.IntegerType()),
        types.StructField("EXAMPLE_COL_6", types.IntegerType()),
        types.StructField("EXAMPLE_COL_7", types.DateType()),
        types.StructField("EXAMPLE_COL_8", types.DateType()),
        types.StructField("EXAMPLE_COL_9", types.DateType())
    ]
)

file2_schema = types.StructType(
    [
        types.StructField("ID", types.IntegerType()),
        types.StructField("EXAMPLE_COL_1", types.IntegerType()),
        types.StructField("EXAMPLE_COL_2", types.StringType()),
        types.StructField("EXAMPLE_COL_3", types.StringType()),
        types.StructField("EXAMPLE_COL_4", types.IntegerType()),
        types.StructField("EXAMPLE_COL_5", types.IntegerType()),
        types.StructField("EXAMPLE_COL_6", types.IntegerType()),
        types.StructField("EXAMPLE_COL_7", types.DateType()),
        types.StructField("EXAMPLE_COL_8", types.DateType()),
        types.StructField("EXAMPLE_COL_9", types.DateType())
    ]
)

In [ ]:
session.sql("TRUNCATE TABLE IF EXISTS FILE1_EXAMPLE_TABLE").collect()
session.sql("TRUNCATE TABLE IF EXISTS FILE2_EXAMPLE_TABLE").collect()
session.sql("TRUNCATE TABLE IF EXISTS FILE3_EXAMPLE_TABLE").collect()

In [ ]:
file1 = session.read \
    .option("FIELD_DELIMITER", ',') \
    .option("SKIP_HEADER", 1) \
    .option("ON_ERROR", "CONTINUE") \
    .schema(file1_schema).csv("@MY_FIRST_STAGE/file1.csv.gz") \
    .copy_into_table("FILE1_EXAMPLE_TABLE")


file2 = session.read \
    .option("FIELD_DELIMITER", '|') \
    .option("SKIP_HEADER", 1) \
    .option("ON_ERROR", "CONTINUE") \
    .schema(file2_schema).csv("@MY_FIRST_STAGE/file2.txt.gz") \
    .copy_into_table("FILE2_EXAMPLE_TABLE")

file3 = session.read.parquet("@MY_FIRST_STAGE/file3.parquet")
file3.copy_into_table("FILE3_EXAMPLE_TABLE")

In [ ]:
session.table("FILE1_EXAMPLE_TABLE")
session.table("FILE2_EXAMPLE_TABLE")
session.table("FILE3_EXAMPLE_TABLE")